# Load Libraries

In [ ]:
import sys
sys.path.append("../input/d/ironluu/swintool/")
sys.path.append("../input/tez-lib/")
sys.path.append("../input/timmmaster/")
from pytorchtools import EarlyStopping
fac = 0.9
patience = 3
lr_patience, lr_factor = 0, 0.35

In [ ]:
import argparse
import math
import tez
import timm
import albumentations
import os
import time
from PIL import Image
from torch.utils.data import Dataset
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
from utils_paw import read_split_data, train_one_epoch, evaluate
from model import swin_large_patch4_window12_384_in22k as create_model
import transformers

# class args:
#     batch_size = 16
#     image_size = 384

def f(x):
    return (561*94.34 + (9912-561-666)*x + 666*8.07)/9912

def sigmoid(x):
    if x<0:
        return math.exp(x) / (1 + math.exp(x))
    return 1 / (1 + math.exp(-x))

def revised(x):
    return (1-fac)/2+fac*x

def inv(x):
    y = (x-(1-fac)/2)/fac
    return y
    
test_aug = None
def lr_schedule_cosine(lr_min, lr_max, per_epochs):
    def compute(epoch):
        return lr_min + 0.5 * (lr_max - lr_min) * (1 + np.cos(epoch / per_epochs * np.pi))
    return compute

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
def draw(oof, oof_true):
    print("bias: ",np.mean(oof)-np.mean(oof_true))
    print("rmse: ",np.sqrt(mean_squared_error(oof_true,oof)))
    print("r2: ",r2_score(oof_true,oof))
    print(max(oof),min(oof))
    plt.hist(oof_true,bins=100)
    plt.hist(oof,bins=100)
    plt.plot([np.mean(oof)]*40,range(40))
    plt.plot([np.mean(oof_true)]*40,range(40))
    plt.show()

# Define Swim Model and Swim Dataset

In [ ]:
class PawpularDataset(Dataset):
    def __init__(self, images_paths, dense_features, images_class, transform, augmentations):
        self.images_paths = images_paths
        self.dense_features = dense_features
        self.images_class = images_class  # torch.tensor(images_class, dtype=torch.float)
        self.transform = transform
        self.augmentations = augmentations

    def __len__(self):
        return len(self.images_paths)

    def __getitem__(self, item):
        image = Image.open(self.images_paths[item])

        if self.transform is not None:
            image = self.transform(image)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]

        # image = np.transpose(image, (2, 0, 1)).astype(np.float32)

        features = self.dense_features[item, :]
        label = self.images_class[item]

        return image, label, features
        # return {
        #     "image": torch.tensor(image, dtype=torch.float),
        #     "features": torch.tensor(features, dtype=torch.float),
        #     "label": torch.tensor(label, dtype=torch.float),
        # }

    @staticmethod
    def collate_fn(batch):
        # 官方实现的default_collate可以参考
        # https://github.com/pytorch/pytorch/blob/67b7e751e6b5931a9f45274653f4f653a4e6cdf6/torch/utils/data/_utils/collate.py
        images, labels, _ = tuple(zip(*batch))
        # features=torch.tensor(features, dtype=torch.float)

        images = torch.stack(images, dim=0)
        labels = torch.as_tensor(labels)
        return images, labels, _


class PawpularModel(tez.Model):
    def __init__(self, args):
        super().__init__()
#         self.model = create_model(num_classes=128).to(args.device)
        #.to(
            #torch.device(args.device if torch.cuda.is_available() else "cpu"))  # args.num_classes
        self.model = timm.create_model("swin_large_patch4_window12_384", pretrained=False, in_chans=3).to(args.device)
        self.model.head = nn.Linear(self.model.head.in_features, 128) .to(args.device)# 重置为128
        self.dropout = nn.Dropout(0.1).to(args.device)
        self.dense1 = nn.Linear(140, 64).to(args.device)
#         self.relu = nn.ReLU().to(args.device)
        self.dense2 = nn.Linear(64, 1).to(args.device)

    def forward(self, image, features, images_class=None):
        x1 = self.model(image)
        x = self.dropout(x1)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
#         x = self.relu(x)
        x = self.dense2(x)

        #x = torch.cat([x, x1, features], dim=1)
        return x, 0, {}

# Import RAPIDS

In [ ]:
def main(args):
    device = torch.device(args.device if torch.cuda.is_available() else "cpu")
    lr = args.lr
    if os.path.exists("./weights") is False:
        os.makedirs("./weights")

    tb_writer = SummaryWriter()

    df = pd.read_csv('../input/pawpular-folds/train_10folds.csv')#.head(100)
#     df = df[(df['Pawpularity'] < 100)&(df['Pawpularity'] >3)]
    print('Train shape:', df.shape)
    final = []
    for fold_ in range(10):
        args.lr = lr
        if fold_ != 8 and fold_ != 9:
            continue
        print('#' * 25)
        print('### FOLD', fold_)
        print('#' * 25)

        # pawmodel = PawpularModel(args)
        
        df_test = pd.read_csv("../input/petfinder-pawpularity-score/test.csv")#.head()
        # map(lambda x: x - 1, df_test['Pawpularity'])
        test_img_paths = [f"../input/petfinder-pawpularity-score/test/{x}.jpg" for x in df_test["Id"].values]

        df_valid = df[df.kfold == fold_].reset_index(drop=True)#.head(10)

        df_valid['norm_score'] = df_valid['Pawpularity']/100
        if args.re:
            df_valid['norm_score'] = df_valid['norm_score'].map(revised)

        valid_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_valid["Id"].values]

        dense_features = [
            'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
        ]

        df_train = df[df.kfold != fold_].reset_index(drop=True)#.head(10)
        if args.data:
            df_train = df_train[(df_train.Pawpularity>15)&(df_train.Pawpularity<80)].reset_index(drop=True)
        df_train['norm_score'] = df_train['Pawpularity']/100
        if args.re:
            df_train['norm_score'] = df_train['norm_score'].map(revised)
        train_img_paths = [f"../input/petfinder-pawpularity-score/train/{x}.jpg" for x in df_train["Id"].values]

        # train_images_path, train_images_label, val_images_path, val_images_label = read_split_data(args.data_path)

        img_size = 384
        data_transform = {
            "train": transforms.Compose([#transforms.RandomResizedCrop(img_size),
                                         transforms.Resize([img_size,img_size]),#transforms.CenterCrop(img_size),
                                         transforms.RandomHorizontalFlip(),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
            "val": transforms.Compose([#transforms.Resize(int(img_size )), #* 1.143
#                                        transforms.CenterCrop(img_size),
                                       transforms.Resize([img_size,img_size]),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])}

        # 实例化训练数据集
        train_dataset = PawpularDataset(
            images_paths=train_img_paths,
            dense_features=df_train[dense_features].values,
            images_class=df_train['norm_score'],#.values,  # list(map(gettarget, df_train['Pawpularity'].values)),
            transform=data_transform["train"],
            augmentations=test_aug
        )

        # 实例化验证数据集
        val_dataset = PawpularDataset(
            images_paths=valid_img_paths,
            dense_features=df_valid[dense_features].values,
            images_class=df_valid['norm_score'],#.values,  # list(map(gettarget, df_valid['Pawpularity'].values)),
            transform=data_transform["val"],
            augmentations=test_aug
        )

        nw = batch_size = args.batch_size
        nw2 = 8
#         nw = min([os.cpu_count(), batch_size if batch_size > 1 else 0, 8])  # number of workers
        print('Using {} dataloader workers every process'.format(nw))
        train_loader = torch.utils.data.DataLoader(train_dataset,
                                                   batch_size=batch_size,
                                                   shuffle=True,
                                                   pin_memory=True,
                                                   num_workers=nw,
                                                   collate_fn=train_dataset.collate_fn)

        val_loader = torch.utils.data.DataLoader(val_dataset,
                                                 batch_size=2*nw2,
                                                 shuffle=False,
                                                 pin_memory=True,
                                                 num_workers=2*nw2,
                                                 collate_fn=val_dataset.collate_fn)

        pawmodel = PawpularModel(args)
        model = pawmodel#.model
#         pawmodel.load(f"../input/paw-models/model_f{fold_}.bin", device="cuda", weights_only=True)
        
        
#         args.weights = f"../input/paw-models/model_f{fold_}.bin"
#         args.weights = f"../input/clearwei/weights/modelfold{fold_}-E.pth"
        if args.weights != "":
            assert os.path.exists(args.weights), "weights file: '{}' not exist.".format(args.weights)
            weights_dict = torch.load(args.weights, map_location=device)["model"]
            # 删除有关分类类别的权重
            for k in list(weights_dict.keys()):
                if "head" in k or "dense" in k: #  
                    del weights_dict[k]
                    continue
#                 if "model." in k:
#                     weights_dict[k[6:]]=weights_dict[k]
#                     del weights_dict[k]
            
            print(model.model.load_state_dict(weights_dict, strict=False))#

        feature_params = []
        output_params = []
        if args.freeze_layers:
            for name, para in model.named_parameters():
                # 除head外，其他权重全部冻结
                if "model" in name:
                    para.requires_grad_(False)
                else:
                    print("training {}".format(name))
        else:
            for name, para in model.named_parameters():
                # 除head外
                if "model" in name and "head" not in name:
                    feature_params.append(para)
                else:
                    output_params.append(para)

        # head Linear

        pg = [p for p in model.parameters() if p.requires_grad]
        scheduler=lr_schedule_cosine(args.final*args.lr,1*args.lr,args.cos)
#         scheduler = CosineScheduler(args.cos, warmup_steps=0, base_lr=1*args.lr, final_lr=0.01*args.lr)
        early_stopping = EarlyStopping(patience, verbose=False, path="./weights/modelfold{}-E.pth".format(fold_))
        for epoch in range(args.epochs):
            
            if epoch<=args.cos:
                args.lr = scheduler(epoch)
#             optimizer = optim.AdamW(pg, lr=args.lr)#, momentum=args.momentum
#             optimizer = optim.AdamW([{'params': feature_params},
#                                  {'params': output_params, 'lr': args.lr*args.multi}], #  * 10, 'weight_decay': 5e-2
#                                 lr=args.lr)
            optimizer = optim.SGD([{'params': feature_params, 'momentum':args.momentum2},
                                 {'params': output_params, 'lr': args.lr*args.multi}], #  * 10, 'weight_decay': 5e-2
                                lr=args.lr, momentum=args.momentum)
#             if epoch >3:
#                 optimizer = optim.SGD(pg, lr=args.lr, momentum=0.99)
#             else:
#                 optimizer = optim.SGD([{'params': feature_params, 'momentum':0.999},
#                                  {'params': output_params, 'lr': args.lr*1}], #  * 10, 'weight_decay': 5e-2
#                                 lr=args.lr, momentum=0.9)
            # train
            train_loss, train_acc, _ = train_one_epoch(model=model,
                                                    optimizer=optimizer,
                                                    data_loader=train_loader,
                                                    device=device,
                                                    epoch=epoch, cutmix_prob = args.cut,
                                                    mixup_prob=0.6, alpha = 0.8,
                                                    lossf=True, bce=True, clip=args.clip,warm=args.warm, factor=1/args.factor)

            # validate
            val_loss, val_acc, pred_list = evaluate(model=model,
                                         data_loader=val_loader,
                                         device=device,
                                         epoch=epoch, lossf=True, bce=True, factor=1/args.factor)

            tags = ["train_loss", "train_mse", "val_loss", "val_mse", "learning_rate"]
            tb_writer.add_scalar(tags[0], train_loss, epoch)
            tb_writer.add_scalar(tags[1], train_acc, epoch)
            tb_writer.add_scalar(tags[2], val_loss, epoch)
            tb_writer.add_scalar(tags[3], val_acc, epoch)
            tb_writer.add_scalar(tags[4], optimizer.param_groups[0]["lr"], epoch)
            
            pred_list = [sigmoid(x) for x in pred_list]
            if args.re:
                pred_list = [inv(x) for x in pred_list]
            pred_list = [x * 100 for x in pred_list]
            draw(pred_list, df_valid['Pawpularity'])
            
            if epoch == args.epochs-1:
                torch.save(model.state_dict(), "./weights/modelfold{}-{}.pth".format(fold_, epoch))
#                 test_predictions = model.predict(val_dataset, batch_size=2*batch_size, n_jobs=-1)
#                 final_test_predictions = []
#                 for preds in test_predictions: #tqdm
#                     final_test_predictions.extend(preds[:,:1].ravel().tolist())
                final += pred_list
                
            early_stopping(val_acc, model)
            # 若满足 early stopping 要求
            if early_stopping.early_stop:
                print("Early stopping")
                # 结束模型训练
                break
    return final

# Infer Test and OOF
In version 1 of this notebook, we extract train embeddings and train RAPIDS SVR heads. (Click version 1 to see this). In later versions and during Kaggle submit, we load these saved RAPIDS SVR fold models and just infer data (without training anything).

In [ ]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_classes', type=int, default=100)
    parser.add_argument('--epochs', type=int, default=8)
    parser.add_argument('--cos', type=int, default=3)
    parser.add_argument('--momentum', type=int, default=0.9)
    parser.add_argument('--momentum2', type=int, default=0.9)
    parser.add_argument('--batch-size', type=int, default=1)
    parser.add_argument('--warm', type=int, default=100)
    parser.add_argument('--lr', type=float, default=1e-4) # 0.0001
    parser.add_argument('--cut', type=float, default=0)
    parser.add_argument('--clip', type=float, default=20.)
    parser.add_argument('--multi', type=float, default=5)
    parser.add_argument('--re', type=bool, default=True)
    parser.add_argument('--data', type=bool, default=False)
    parser.add_argument('--factor', type=float, default=fac)
    parser.add_argument('--final', type=float, default=0.05)
    # 数据集所在根目录
    # http://download.tensorflow.org/example_images/flower_photos.tgz
    # parser.add_argument('--data-path', type=str,
    #                     default="../data/flower_photos")

    # 预训练权重路径，如果不想载入就设置为空字符
    parser.add_argument('--weights', type=str, default='../input/swinweight/swin_large_patch4_window12_384_22k.pth',
                        help='initial weights path')
    # 是否冻结权重
    parser.add_argument('--freeze-layers', type=bool, default=False)
    parser.add_argument('--device', default='cuda:0', help='device id (i.e. 0 or 0,1 or cpu)')

    args = parser.parse_args(args=[])

    test_predictions = main(args)

# Compute CV Score
Below we compute the overall CV RSME scores of just the NN head, just the SVR head, and an ensemble of 50% NN and 50% SVR heads. Then we plot all ensemble weights to find the optimal weights for NN head and SVR heads.

# Make Submission CSV
We make a submission csv using an ensemble of both heads. We use the optimal ensemble weights that we discovered above.

In [ ]:
# super_final_predictions = np.mean(np.column_stack(super_final_predictions), axis=1)
# # super_final_predictions2 = np.mean(np.column_stack(super_final_predictions2), axis=1)
# super_final_predictions3 = np.mean(np.column_stack(super_final_predictions3), axis=1)
# df_test["Pawpularity"] = (1-best_w)*super_final_predictions + best_w*super_final_predictions3
# df_test = df_test[["Id", "Pawpularity"]]
# df_test.to_csv("submission.csv", index=False)
# df_test.head()